In [1]:
using Pkg; Pkg.activate("/home/dhairyagandhi96/temp/model-zoo/script/.."); Pkg.status();

include("data.jl")
using Flux, Statistics
using Flux: onehot, onehotbatch, throttle, crossentropy, reset!, onecold

const epochs = 30

    Status `~/temp/model-zoo/Project.toml`
  [1520ce14]   AbstractTrees v0.2.1
  [fbb218c0] ↑ BSON v0.2.3 ⇒ v0.2.4
  [54eefc05]   Cascadia v0.4.0
  [8f4d0f93]   Conda v1.3.0
  [864edb3b] ↑ DataStructures v0.17.0 ⇒ v0.17.5
  [31c24e10] ↑ Distributions v0.21.3 ⇒ v0.21.5
  [587475ba]   Flux v0.9.0
  [708ec375]   Gumbo v0.5.1
  [b0807396]   Gym v1.1.3
  [cd3eb016] ↑ HTTP v0.8.6 ⇒ v0.8.7
  [6218d12a]   ImageMagick v0.7.5
  [916415d5]   Images v0.18.0
  [e5e0dc1b]   Juno v0.7.2
  [ca7b5df7]   MFCC v0.3.1
  [dbeba491] + Metalhead v0.4.0 #c4d1eba (https://github.com/FluxML/Metalhead.jl.git)
  [91a5bcdd] ↑ Plots v0.26.3 ⇒ v0.27.0
  [2913bbd2]   StatsBase v0.32.0
  [98b73d46]   Trebuchet v0.1.0
  [8149f6b0] ↑ WAV v1.0.2 ⇒ v1.0.3
  [e88e6eb3] + Zygote v0.4.1
  [10745b16]   Statistics 
  [4ec0a83e]   Unicode 


30

training data of bit strings from length 2 to 10

In [2]:
train = gendata(2000, 1:10)

2000-element Array{Tuple{Flux.OneHotMatrix{Array{Flux.OneHotVector,1}},Flux.OneHotVector},1}:
 ([false true true false; true false false true], [true, false])    
 ([true false … true false; false true … false true], [true, false])
 ([true false true; false true false], [false, true])               
 ([false true … true false; true false … false true], [true, false])
 ([true true false; false false true], [false, true])               
 ([true true … true false; false false … false true], [false, true])
 ([false true … false false; true false … true true], [true, false])
 ([true true … true false; false false … false true], [false, true])
 ([false; true], [false, true])                                     
 ([true false … false false; false true … true true], [true, false])
 ⋮                                                                  
 ([true false … false false; false true … true true], [false, true])
 ([true false … true true; false true … false false], [false, true])
 ([false 

validation data of bit strings of length 10

In [3]:
val = gendata(100, 10)

scanner = LSTM(length(alphabet), 20)
encoder = Dense(20, length(alphabet))

function model(x)
    state = scanner.(x.data)[end]
    reset!(scanner)
    softmax(encoder(state))
end

loss(x, y) = crossentropy(model(x), y)
batch_loss(data) = mean(loss(d...) for d in data)

opt = ADAM()
ps = params(scanner, encoder)
evalcb = () -> @show batch_loss(val)

for i=1:epochs
    Flux.train!(loss, ps, train, opt, cb=throttle(evalcb, 10))
end

batch_loss(val) = 0.71469086f0 (tracked)
batch_loss(val) = 0.6927051f0 (tracked)
batch_loss(val) = 0.69424677f0 (tracked)
batch_loss(val) = 0.43826458f0 (tracked)
batch_loss(val) = 0.0069964584f0 (tracked)
batch_loss(val) = 0.001525304f0 (tracked)
batch_loss(val) = 0.00042881002f0 (tracked)
batch_loss(val) = 0.0001355765f0 (tracked)
batch_loss(val) = 4.522074f-5 (tracked)
batch_loss(val) = 1.5424232f-5 (tracked)
batch_loss(val) = 5.322735f-6 (tracked)
batch_loss(val) = 1.8441721f-6 (tracked)
batch_loss(val) = 6.425387f-7 (tracked)
batch_loss(val) = 2.2292144f-7 (tracked)
batch_loss(val) = 5.960465f-8 (tracked)
batch_loss(val) = 1.0728838f-8 (tracked)
batch_loss(val) = 3.5762793f-9 (tracked)
batch_loss(val) = 1.1920931f-9 (tracked)
batch_loss(val) = -0.0f0 (tracked)
batch_loss(val) = -0.0f0 (tracked)
batch_loss(val) = -0.0f0 (tracked)
batch_loss(val) = -0.0f0 (tracked)
batch_loss(val) = -0.0f0 (tracked)
batch_loss(val) = -0.0f0 (tracked)
batch_loss(val) = -0.0f0 (tracked)
batch_loss(val

sanity test

In [4]:
tx = map(c -> onehotbatch(c, alphabet), [
    [false, true], # 01 -> 1
    [true, false], # 10 -> 1
    [false, false], # 00 -> 0
    [true, true]]) # 11 -> 0
[onecold(model(x)) - 1 for x in tx] |> println

[1, 1, 0, 0]


Try running the model on strings of length 50.

Even though the model has only been trained with
much shorter strings, it has learned the
parity function and will accurate on longer strings.

In [5]:
function t50()
    l = batch_loss(gendata(1000, 50))
    println(l)
end

t50 (generic function with 1 method)